In [ ]:
# Copyright 2022-2023 Sony Semiconductor Solutions Corp. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Delete "**Edge Application**" on "**Console for AITRIOS**"

This notebook explains the workflow for deleting "**Edge Application**" from "**Console**". <br>
<br>
Instructions are described in [README.md](./README.md).

## Imports

In [ ]:
import errno
import json
from pathlib import Path

import jsonschema
from console_access_library.client import Client

## Load Configurations

Load the configuration file and set the variables.

In [ ]:
def validate_symlink(path: Path):
    if path.is_symlink():
        raise OSError(
            errno.ELOOP,
            "Symbolic link is not supported. Please use real folder or file",
            f"{path}",
        )


configuration_path = Path("./configuration.json")
validate_symlink(configuration_path)

# Load configuration file
with open(configuration_path, "r") as f:
    json_load = json.load(f)

configuration_schema_path = Path("./configuration_schema.json")
validate_symlink(configuration_schema_path)

# Load configuration schema file
with open(configuration_schema_path, "r") as f:
    json_schema = json.load(f)

# Validate configuration
jsonschema.validate(json_load, json_schema)

## Load an instance of "**Console Access Library**" Client

Run this cell to restore instance variable 'client_obj' created on a notebook of Set up "**Console Access Library**".

In [ ]:
%store -r
client_obj: Client = client_obj

## Delete "**Edge Application**"

In [ ]:
# Get an instance of Deployment API
deployment_obj = client_obj.get_deployment()

# Call an API for deleting Vision and Sensing Application
try:
    response = deployment_obj.delete_device_app(
        app_name=json_load["app_name"], version_number=json_load["version_number"]
    )
except Exception as e:
    # EXCEPTION
    raise e

if "result" in response and response["result"] != "SUCCESS":
    # ERROR
    raise ValueError("ERROR", response)

# SUCCESS
print(
    "Start to delete the Vision and Sensing Application."
    + " \n\tapp_name: "
    + json_load["app_name"]
    + "\n\tversion_number: "
    + json_load["version_number"]
)

## Check if the "**Edge Application**" has been deleted

In [ ]:
# Flag for deleting check
delete_flag = True

# Call an API to get Vision and Sensing Application info
try:
    response = deployment_obj.get_device_apps()
except Exception as e:
    # EXCEPTION
    raise e

if "result" in response and response["result"] != "SUCCESS":
    # ERROR
    raise ValueError("ERROR", response)

# SUCCESS
# Check deleting
apps = response.get("apps", [])
for app in apps:
    if "name" in app and app["name"] == json_load["app_name"]:
        versions = app.get("versions", [])
        for version in versions:
            if (
                "version" in version
                and version["version"] == json_load["version_number"]
            ):
                delete_flag = False
                break
        if not delete_flag:
            break
# View results
if delete_flag:
    print(
        "Deleting Vision and Sensing Application is completed."
        + " \n\tapp_name: "
        + json_load["app_name"]
        + "\n\tversion_number: "
        + json_load["version_number"]
    )
else:
    raise Exception(
        "Deleting Vision and Sensing Application is failed."
        + " \n\tapp_name: "
        + json_load["app_name"]
        + "\n\tversion_number: "
        + json_load["version_number"]
    )